<a href="https://colab.research.google.com/github/arungovindm/word2vec-skipgram/blob/arch-1/word2vec_skipgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import nltk
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
nltk.download('reuters')
from nltk.corpus import reuters
t=reuters.fileids()

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [3]:
from nltk.corpus import stopwords
nltk.download('stopwords')
rem = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
train=[]
test=[]
for a in t:
    if a.startswith('training'):
        train.append(a)
    else:
        test.append(a)

In [0]:
corpus=reuters.words(train)
fdist = nltk.FreqDist(corpus)

In [0]:
filtered_corpus = []
for w in corpus:
  if not (w.isdigit() or w in rem or len(w)<3):
    filtered_corpus.append(w)

In [0]:
vocab_size=len(nltk.FreqDist(filtered_corpus))
d=300 #dimensionality
power =0.75

In [10]:
Z=0
vocab = nltk.FreqDist(filtered_corpus)
for a,b in vocab.items():
  Z=Z+b**power
print(Z)

195801.74701170594


In [0]:
vocab = nltk.FreqDist(filtered_corpus)

In [0]:
word_idx = dict()
idx_word=dict()
dict_count = dict()
i=1
for word, count in vocab.items():
  word_idx[word]=i
  idx_word[i]=word
  dict_count[i]=count
  i=i+1

In [0]:
labelled_corpus=[]
for word in filtered_corpus:
  labelled_corpus.append(word_idx[word])

In [0]:
del corpus, filtered_corpus

In [0]:
window_size = 2 #context window size
neg=5 #number of negative samples
target=[]
context=[]

In [0]:
for i in range(len(labelled_corpus)):
  for j in range(window_size):
    if i+j+1 < len(labelled_corpus):
      context.append(labelled_corpus[i+j+1])
      target.append(labelled_corpus[i])
    if i-j-1 >=0:
      context.append(labelled_corpus[i-j-1])
      target.append(labelled_corpus[i])

In [0]:
dict_prob=dict()
for idx,count in dict_count.items():
  dict_prob[idx]=(dict_count[idx]**power)/Z

In [0]:
def unigram_sampling(dct):
    rand_val = np.random.random()
    total = 0
    for k, v in dct.items():
        total += v
        if rand_val <= total:
            return k
    assert False, 'unreachable'

In [0]:
def create_train_instance(idx,neg):  #target word, context word and neg negative context samples
  train_instance=[]
  train_instance.append(target[idx])
  train_instance.append(context[idx])
  i=0
  for i in range(neg):
    train_instance.append(unigram_sampling(dict_prob))
  return train_instance

In [0]:
# building the neural net with tensorflow

#model parameteres
b=tf.Variable(tf.zeros(vocab_size,1))
W=tf.Variable(tf.random_uniform((vocab_size,d),-1,1))


#input
w=tf.placeholder(tf.int32, (neg+2,1))

#hidden layer
H1=tf.reshape(tf.nn.embedding_lookup(W,w),[neg+2,d])
v_w=tf.reshape(tf.nn.embedding_lookup(H1,[0]), [1,d])
C = tf.reshape(tf.nn.embedding_lookup(H1,w[1:]),[neg+1,d])
H2=tf.matmul(v_w,tf.transpose(C))

#loss
#prediction = tf.nn.sigmoid(tf.matmul(tf.transpose(v_w),v_c))
#loss = -1*tf.log(prediction)
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [48]:
H2.get_shape().as_list()  # [2, 2, 3]


[1, 6]

In [0]:
sesh = tf.Session()
sesh.run(tf.initialize_all_variables())
sesh.run(H1, feed_dict={w:create_train_instance})

for i in range(1000):
  batch_v,batch_w = 


In [26]:
a=[[1,2,1],[0,0,0],[7,8,9]]
b,c=np.split(a, [1,np.shape(a)[0]-1],0)

ValueError: ignored

In [29]:
create_train_instance(1,neg)[1:]

[3, 1282, 13110, 10849, 4926, 2072]